In [56]:
#lol y'a pas de module pour le torus on est dans la merde

In [57]:
# si k E Tp , il existe un entier r E [0;p] tq k = r/p
# on peut donc considérer k comme (r,p) ou juste comme r jsp

In [58]:
def AddInTore(p,a,b) :
    return (a+b)%p

In [59]:
def MultExtTore(p,n,x):
    if(n>=0):
        cpt = 0
        for i in range(n):
            cpt = AddInTore(p,cpt,x)
        return cpt
    return MultExtTore(p,-n,-x)

In [151]:
def TLWEncrypt(p,q,n,s,sigma,m):
    a=[]
    for i in range(n):
        a.append(randint(0,q-1))
    e = RealDistribution("gaussian",sigma).get_random_element()
    e = round(q*e)
    while(abs(e/q)>=1/(2*p)):
        e = RealDistribution("gaussian",sigma).get_random_element()
        e = round(q*e)
    #print("e= ",e)
    c = AddInTore(q,m*q/p,e)
    for i in range(n):
        c = AddInTore(q,c,MultExtTore(q,s[i],a[i]))
    return a+[c]

In [61]:
def TLWEDecrypt(p,q,n,s,c):
    a = c[:-1]
    m = c[-1]
    for i in range(n):
        m = AddInTore(q,m,MultExtTore(q,-s[i],a[i]))
    #print(m/q)
    return round(m*p/q)%p

In [62]:
p = 16
q = 64
n = 5
s = [0,1,1,0,0]
sigma = 0.001

m = 10

c = TLWEncrypt(p, q, n, s, sigma, m)
print("c =",c)

mmaybe = TLWEDecrypt(p, q, n, s, c)
print("m =",mmaybe)

c = [42, 11, 47, 13, 13, 34]
m = 10


In [63]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt = 0
for i in range(10000):
    
    m = randint(0,p-1)


    c = TLWEncrypt(p, q, n, s, sigma, m)

    mmaybe = TLWEDecrypt(p, q, n, s, c)
    if(m==mmaybe):
        cpt = cpt +1
print(cpt)        

10000


In [64]:
def AddCiph(q,n,c1,c2):
    return [AddInTore(q,c1[i],c2[i]) for i in range(n+1)]

In [73]:
p = 2**9
q = 2**16
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m1 = randint(0,p-1)
    m2 = randint(0,p-1)

    c1,e1 = TLWEncrypt(p, q, n, s, sigma, m1)
    c2,e2 = TLWEncrypt(p, q, n, s, sigma, m2)

    m = TLWEDecrypt(p, q, n, s, AddCiph(q,n,c1,c2))
    if(m==AddInTore(p,m1,m2)):
        cpt1 = cpt1 +1
    elif(abs((e1+e2)/q)>=1/(2*p)):
        cpt2 = cpt2 + 1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 ==1000)

752
248
True


In [26]:
def MultCiph(q,n,K,c):
    return [MultExtTore(q,K,c[i]) for i in range(n+1)]

In [76]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m = randint(0,p-1)

    c,e = TLWEncrypt(p, q, n, s, sigma, m)
    
    K = randint(1,100)
    mmaybe = TLWEDecrypt(p, q, n, s, MultCiph(q,n,K,c))
    
    if(mmaybe==MultExtTore(p,K,m)):
        cpt1 +=1
    elif(abs((K*e)/q)>=1/(2*p)):
        cpt2 +=1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 == 1000)

648
352
True


In [122]:
def TrueGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = 1/(B^(j+1))
    return G
#G is the real gadget matrix

In [119]:
def UsedGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = q/(B^(j+1))
    return G
#G is the gadhet matrix of the numerator of the real gadget matrix with q for denominator

In [144]:
def AddInMatr(q,M,L):
    return [[AddInTore(q,M[i][j],L[i][j]) for j in range(len(M[0]))] for i in range(len(M))]

In [145]:
#Works with M a matrix with coeff in our Tq
def MultExtMatr(q,K,M):
    return [[MultExtTore(q,K,M[i][j]) for j in range(len(M[0]))] for i in range(len(M))] 

In [158]:
def TGSWEncrypt(q,B,l,n,m,s):
    sigma = 0.001
    Z = [TLWEncrypt(B**l,q,n,s,sigma,0) for i in range(n+1)]
    #print(Z)
    return AddInMatr(q,Z,MultExtMatr(q,m,UsedGadgetMatrix(q,B,l,n))),MultExtMatr(q,m,UsedGadgetMatrix(q,B,l,n))

In [161]:
a,b = TGSWEncrypt(64,2,3,6,7,[0,1,1,1,0,1])

In [160]:
b

[[32, 0, 0, 0, 0, 0, 0],
 [48, 0, 0, 0, 0, 0, 0],
 [56, 0, 0, 0, 0, 0, 0],
 [0, 32, 0, 0, 0, 0, 0],
 [0, 48, 0, 0, 0, 0, 0],
 [0, 56, 0, 0, 0, 0, 0],
 [0, 0, 32, 0, 0, 0, 0],
 [0, 0, 48, 0, 0, 0, 0],
 [0, 0, 56, 0, 0, 0, 0],
 [0, 0, 0, 32, 0, 0, 0],
 [0, 0, 0, 48, 0, 0, 0],
 [0, 0, 0, 56, 0, 0, 0],
 [0, 0, 0, 0, 32, 0, 0],
 [0, 0, 0, 0, 48, 0, 0],
 [0, 0, 0, 0, 56, 0, 0],
 [0, 0, 0, 0, 0, 32, 0],
 [0, 0, 0, 0, 0, 48, 0],
 [0, 0, 0, 0, 0, 56, 0],
 [0, 0, 0, 0, 0, 0, 32],
 [0, 0, 0, 0, 0, 0, 48],
 [0, 0, 0, 0, 0, 0, 56]]

In [162]:
b

[[32, 0, 0, 0, 0, 0, 0],
 [48, 0, 0, 0, 0, 0, 0],
 [56, 0, 0, 0, 0, 0, 0],
 [0, 32, 0, 0, 0, 0, 0],
 [0, 48, 0, 0, 0, 0, 0],
 [0, 56, 0, 0, 0, 0, 0],
 [0, 0, 32, 0, 0, 0, 0],
 [0, 0, 48, 0, 0, 0, 0],
 [0, 0, 56, 0, 0, 0, 0],
 [0, 0, 0, 32, 0, 0, 0],
 [0, 0, 0, 48, 0, 0, 0],
 [0, 0, 0, 56, 0, 0, 0],
 [0, 0, 0, 0, 32, 0, 0],
 [0, 0, 0, 0, 48, 0, 0],
 [0, 0, 0, 0, 56, 0, 0],
 [0, 0, 0, 0, 0, 32, 0],
 [0, 0, 0, 0, 0, 48, 0],
 [0, 0, 0, 0, 0, 56, 0],
 [0, 0, 0, 0, 0, 0, 32],
 [0, 0, 0, 0, 0, 0, 48],
 [0, 0, 0, 0, 0, 0, 56]]